# Advanced Transformations: Superheroes

# Part I

Clean the files and combine them into one final DataFrame.

## Import Packages

In [114]:
## Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Importing the OS and JSON Modules
import os,json

# Suppress PerformanceWarning (PW) for "OHE" of superpowers
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

## Load Data

In [48]:
# Load superhero_info.csv
df_info = pd.read_csv('Data/superhero_info.csv')
print(df_info.info())
df_info.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB
None


,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"


In [49]:
# Load superhero_powers.csv
df_powers = pd.read_csv('Data/superhero_powers.csv')
print(df_powers.info())
df_powers.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB
None


,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."


## Transformations

### Split Hero|Publisher
Separate a string column into multiple columns

In [50]:
# Display Hero|Publisher column
df_info['Hero|Publisher'].head(2)

0            A-Bomb|Marvel Comics
1    Abe Sapien|Dark Horse Comics
Name: Hero|Publisher, dtype: object

In [51]:
# Split column on "|" into two columns
df_info['Hero|Publisher'].str.split("|", expand = True)

,0,1
0,A-Bomb,Marvel Comics
1,Abe Sapien,Dark Horse Comics
2,Abin Sur,DC Comics
3,Abomination,Marvel Comics
4,Absorbing Man,Marvel Comics
...,...,...
458,Yellowjacket,Marvel Comics
459,Yellowjacket II,Marvel Comics
460,Yoda,George Lucas
461,Zatanna,DC Comics


In [52]:
# Save column split back into df_info
df_info[['Hero','Publisher']] = df_info['Hero|Publisher'].str.split("|", expand = True)
df_info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [53]:
# Drop original 'Hero|Publisher' column, as no longer needed
df_info = df_info.drop(columns=['Hero|Publisher'])
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


### Separate Measurements Dictionary
* Convert "string" dictionaries back into actual dictionaries
* Unpack dictionaries into separate columns
* Convert unit measurements into unit-less integer values

#### Convert "string" dictionaries back into actual dictionaries

In [54]:
# Examine single value from 'Measurements' col
measurements = df_info.loc[0,'Measurements']
print(type(measurements))
measurements

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [55]:
# Fix single/double quotes so json.loads can function correctly
measurements = measurements.replace("'",'"')
measurements

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [56]:
# Convert "string" dictionary to actual dictionary
fixed_measurements = json.loads(measurements)
print(type(fixed_measurements))
fixed_measurements

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

In [57]:
# Apply successful method above to entire column
df_info['Measurements'] = df_info['Measurements'].str.replace("'",'"')
df_info['Measurements'] = df_info['Measurements'].apply(json.loads)
df_info['Measurements'].head(3)

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
Name: Measurements, dtype: object

In [58]:
# Check single value to confirm tranformation
test_me = df_info.loc[0, 'Measurements']
print(type(test_me))
test_me

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

#### Unpack dictionaries into separate columns

In [59]:
# Convert all keys in 'Measurements' dictionary into new column
height_weight = df_info['Measurements'].apply(pd.Series)
height_weight

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [60]:
# Concat height_weight with original dataframe
df_info = pd.concat((df_info, height_weight), axis= 1)
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [61]:
# Drop 'Measurements' as no longer needed
df_info = df_info.drop(columns=['Measurements'])
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [62]:
# Check data types for 'Height' & 'Weight' columns 
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Gender      463 non-null    object
 1   Race        463 non-null    object
 2   Alignment   463 non-null    object
 3   Hair color  463 non-null    object
 4   Eye color   463 non-null    object
 5   Skin color  463 non-null    object
 6   Hero        463 non-null    object
 7   Publisher   463 non-null    object
 8   Height      463 non-null    object
 9   Weight      463 non-null    object
dtypes: object(10)
memory usage: 36.3+ KB


#### Convert unit measurements into unit-less integer values

In [63]:
# Convert both 'Height' & 'Weight' columns into unit-less integer values only
df_info[['Weight','Weight_unit']] = df_info['Weight'].str.split(".", expand = True)
df_info[['Height','Height_unit']] = df_info['Height'].str.split(".", expand = True)
df_info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Weight_unit,Height_unit
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203,441,0 kg,0 cm
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191,65,0 kg,0 cm
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185,90,0 kg,0 cm
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203,441,0 kg,0 cm
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193,122,0 kg,0 cm


In [65]:
# Drop unneeded 'Weight_unit' & 'Height_unit' columns
df_info = df_info.drop(columns=['Weight_unit', 'Height_unit'])
df_info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203,441
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191,65


In [69]:
# Check 'Height' & 'Weight' datatypes
df_info[['Height','Weight']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Height  463 non-null    object
 1   Weight  463 non-null    object
dtypes: object(2)
memory usage: 7.4+ KB


In [70]:
# Change both to numeric (integer) values
df_info[['Height','Weight']] = df_info[['Height','Weight']].astype(int)
print(df_info[['Height','Weight']].dtypes)

Height    int64
Weight    int64
dtype: object


### Combine Separate CSV Files into One

In [74]:
# See number of heroes in each of the two files
print(df_powers['hero_names'].info())
print()
print(df_info['Hero'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 667 entries, 0 to 666
Series name: hero_names
Non-Null Count  Dtype 
--------------  ----- 
667 non-null    object
dtypes: object(1)
memory usage: 5.3+ KB
None

<class 'pandas.core.series.Series'>
RangeIndex: 463 entries, 0 to 462
Series name: Hero
Non-Null Count  Dtype 
--------------  ----- 
463 non-null    object
dtypes: object(1)
memory usage: 3.7+ KB
None


In [78]:
# Change 'hero-names' in df_powers to match 'Hero' column name in df_info
df_powers = df_powers.rename(columns={'hero_names':'Hero'})
df_powers.head(2)

,Hero,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."


In [103]:
# Length of each file is different, so need to include unique names in each hero-name column
# Combine files based on name of 'Hero'
df = pd.merge(df_info, df_powers, on='Hero', how='right')
df.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Powers
0,NaN,NaN,NaN,NaN,NaN,NaN,3-D Man,NaN,NaN,NaN,"Agility,Super Strength,Stamina,Super Speed"
1,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,"Accelerated Healing,Durability,Longevity,Super..."
2,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,"Agility,Accelerated Healing,Cold Resistance,Du..."


In [104]:
# Verify row count matches highest row count between the two files: 
print(len(df_info))
print(len(df_powers))
print()
print(len(df))

463
667

675


### One-Hot Encode Powers
* OHE every power into its own column

In [105]:
# Check if values in 'Powers' are lists or actually strings
df.loc[0,'Powers']

'Agility,Super Strength,Stamina,Super Speed'

In [106]:
# Check type for values in 'Powers'
print(type(df.loc[0,'Powers']))

<class 'str'>


In [107]:
# Convert comma-separated string values into a list
df_powers_list = df['Powers'].str.split(",", expand=False)
df_powers_list

0        [Agility, Super Strength, Stamina, Super Speed]
1      [Accelerated Healing, Durability, Longevity, S...
2      [Agility, Accelerated Healing, Cold Resistance...
3                                   [Lantern Power Ring]
4      [Accelerated Healing, Intelligence, Super Stre...
                             ...                        
670               [Flight, Energy Blasts, Size Changing]
671    [Cold Resistance, Durability, Longevity, Super...
672    [Agility, Stealth, Danger Sense, Marksmanship,...
673    [Cryokinesis, Telepathy, Magic, Fire Control, ...
674    [Super Speed, Intangibility, Time Travel, Time...
Name: Powers, Length: 675, dtype: object

In [108]:
# Look at values in df_powers_list
df_powers_list.value_counts().head()

[Intelligence]                  8
[Durability, Super Strength]    5
[Weapon-based Powers]           4
[Teleportation]                 4
[Electrokinesis]                4
Name: Powers, dtype: int64

In [109]:
print(type(df_powers_list))

<class 'pandas.core.series.Series'>


In [110]:
# Exploding the column of lists
exploded = df_powers_list.explode()
exploded.head(3)

0           Agility
0    Super Strength
0           Stamina
Name: Powers, dtype: object

In [111]:
# Save unique values from exploded
cols_to_make = exploded.dropna().unique()
cols_to_make

array(['Agility', 'Super Strength', 'Stamina', 'Super Speed',
       'Accelerated Healing', 'Durability', 'Longevity', 'Camouflage',
       'Self-Sustenance', 'Cold Resistance', 'Underwater breathing',
       'Marksmanship', 'Weapons Master', 'Intelligence', 'Telepathy',
       'Immortality', 'Reflexes', 'Enhanced Sight', 'Sub-Mariner',
       'Lantern Power Ring', 'Invulnerability', 'Animation',
       'Super Breath', 'Dimensional Awareness', 'Flight', 'Size Changing',
       'Teleportation', 'Magic', 'Dimensional Travel',
       'Molecular Manipulation', 'Energy Manipulation', 'Power Cosmic',
       'Energy Absorption', 'Elemental Transmogrification',
       'Fire Resistance', 'Natural Armor', 'Heat Resistance',
       'Matter Absorption', 'Regeneration', 'Stealth', 'Power Suit',
       'Energy Blasts', 'Energy Beams', 'Heat Generation', 'Danger Sense',
       'Phasing', 'Force Fields', 'Hypnokinesis', 'Invisibility',
       'Enhanced Senses', 'Jump', 'Shapeshifting', 'Elasticity',
 

In [115]:
# Create new column for each unique superpower
# Indicate True or False if hero had superpower in original list
for col in cols_to_make:
    df[col] = df['Powers'].str.contains(col)

df.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,...,Weather Control,Omnipresent,Omniscient,Hair Manipulation,Nova Force,Odin Force,Phoenix Force,Intuitive aptitude,Melting,Changing Armor
0,NaN,NaN,NaN,NaN,NaN,NaN,3-D Man,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,False,False
1,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
2,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,False,False,False,False


In [126]:
# Verify that "Wolverine" has the ability "Accelerated Healing"
wolverine_a_healing = df.query("Hero == 'Wolverine'")["Accelerated Healing"]
print(f'Hero Wolverine has accelerated healing? {wolverine_a_healing}')
print(df['Accelerated Healing'].value_counts())
print()

# Verify that "Sylar" has the ability "Melting"
sylar_melting = df.query("Hero == 'Sylar'")["Melting"]
print(f'Villain Sylar has the superpower melting? {sylar_melting}')
df['Melting'].value_counts()

Hero Wolverine has accelerated healing? 663    True
Name: Accelerated Healing, dtype: bool
False    492
True     183
Name: Accelerated Healing, dtype: int64

Villain Sylar has the superpower melting? 599    True
Name: Melting, dtype: bool


False    673
True       2
Name: Melting, dtype: int64

All superpowers have been one-hot encoded for each hero.

# Part II

Use your combined DataFrame to answer a few superhero questions.

Compare the average weight of super heroes who have Super Speed to those who do not.

In [127]:
df.groupby('Super Speed')['Weight'].mean()

Super Speed
False    101.773585
True     129.404040
Name: Weight, dtype: float64

* The average weight of heroes with Super Speed is:    129.4 kg.

* The average weight of heroes without Super Speed is: 101.77 kg.

* Super Speeders are about 28 kg heavier on average than non-Super Speeders.

What is the average height of heroes for each publisher?

In [134]:
df.Publisher.value_counts()

Marvel Comics        297
DC Comics            138
Dark Horse Comics     11
George Lucas           5
Team Epic TV           4
Shueisha               4
Star Trek              2
Unknown                1
Image Comics           1
Name: Publisher, dtype: int64

In [135]:
round(df.groupby('Publisher')['Height'].mean().nlargest(9),2)

Publisher
Image Comics         211.00
Marvel Comics        191.55
DC Comics            181.92
Star Trek            181.50
Team Epic TV         180.75
Unknown              178.00
Dark Horse Comics    176.91
Shueisha             171.50
George Lucas         159.60
Name: Height, dtype: float64

* Yoda and the Ewoks appear to be bringing the average height down for the George Lucas universe...